In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
# Jenny's codes start here
# vaccine_us_county_clean file is pre-cleaned with selected columns from CDC data

vaccine_county = pd.read_csv("Clean_data/vaccine_us_county_clean.csv",low_memory=False)

In [ ]:
vaccine_county_rename = vaccine_county.rename(columns={"Recip_County": "County", "Series_Complete_Pop_Pct": "Vaccination_Complete_Pct","Recip_State":"State"})
vaccine_county_rename.shape

In [ ]:
# Merge US vaccination file with US coord file

us_county = pd.read_csv("Resource/05_us_county_coord.csv",low_memory=False)
us_county_rename = us_county.rename(columns={"fips": "FIPS", "county": "County", "state": "State"})
us_county_coord = us_county_rename[['FIPS','lat','long']]

In [ ]:
vaccine_county_coord = pd.merge(vaccine_county_rename, us_county_coord, on='FIPS')
vaccine_county_coord.shape

In [ ]:
# Merge US vaccination_coord file with SIV file

svi_df = pd.read_csv("Resource/04_siv_county.csv",low_memory=False)
svi_df = svi_df.dropna(subset=['Social Vulnerability Index (SVI)'])
svi_df = svi_df.loc[svi_df['Social Vulnerability Index (SVI)']!=0]
svi_df_rename = svi_df.rename(columns={"FIPS Code": "FIPS", "County Name": "County"})
# svi_df_rename.shape

In [ ]:
svi_df_rename['County'] = svi_df_rename['County'].str.split(',', expand=True)[0]

In [ ]:
vaccine_svi_df = pd.merge(vaccine_county_coord, svi_df_rename, on='FIPS')
vaccine_svi_df = vaccine_svi_df[['Date','FIPS','County_x','State_x','Vaccination_Complete_Pct','Metro_status','Social Vulnerability Index (SVI)','SVI Category','lat','long']] 
vaccine_svi_df = vaccine_svi_df.rename(columns={"County_x": "County","State_x": "State"})
vaccine_svi_df.shape
# SVI has no Puerto Rico data 

In [ ]:
# Merge US vaccination_coord with US election data to add FIPS code

us_election= pd.read_csv("Resource/03_president_county_candidate.csv",low_memory=False)

In [ ]:
us_election_rename = us_election.rename(columns={"state": "State","county":"County"})
us_election_fips = pd.merge(us_election_rename,us_county_rename,how='inner',left_on = ['State','County'], right_on=['State','County'])

# Only keep DEM&REP rows

us_election_fips = us_election_fips.loc[(us_election_fips.party=='DEM')|(us_election_fips.party=='REP')]
us_election_fips.shape

In [ ]:
# Merge US vaccination with US election_fips file

vaccine_election = pd.merge(vaccine_county_rename, us_election_fips, on='FIPS')
vaccine_election_df = vaccine_election[['Date','FIPS','County_x','State_x','Vaccination_Complete_Pct','candidate','party','total_votes','won','lat','long']]
vaccine_election_df = vaccine_election_df.rename(columns={"County_x": "County","State_x": "State"})
vaccine_election_df.shape

In [ ]:
vaccine_svi_df.to_csv("Clean_data/vaccine_svi_df.csv", encoding="utf-8", index=False)

In [ ]:
vaccine_county_coord.to_csv("Clean_data/vaccine_county_coord.csv", encoding="utf-8", index=False)

In [ ]:
vaccine_election_df.to_csv("Clean_data/vaccine_election_df.csv", encoding="utf-8", index=False)

In [ ]:
vaccine_county_rename = vaccine_county.rename(columns={"Recip_County": "County", "Series_Complete_Pop_Pct": "Vaccination_Complete_Pct","Recip_State":"State"})
vaccine_county_rename.head()

In [ ]:
socio_health_df = pd.read_csv("Resource/02_us_county_sociohealth_data.csv",low_memory=False)
socio_health_df_rename = socio_health_df.rename(columns={"fips": "FIPS"})

In [ ]:
socio_health_df_rename

In [ ]:
#need to convert FIPS to string for socio_health_dat
vaccine_county_rename = vaccine_county_rename.astype({"FIPS":str})

#merging the county data with the socio health data
vaccine_socio_df = pd.merge(vaccine_county_rename,socio_health_df_rename, on='FIPS')

In [ ]:
vaccine_socio_df = pd.merge(vaccine_county_rename,socio_health_df_rename,on="FIPS")

In [ ]:
vaccine_socio_df

In [ ]:
vaccine_socio_df.to_csv("Clean_data/vaccine_socio_df.csv", encoding="utf-8", index=False)

## feipeng clean data
## Read Data from two datasets: vaccine_us_county_clean.csv and 02_us_county_sociohealth_data.csv

In [2]:
# Get vaccination data from Clean_data/vaccine_us_county_clean.csv
fy_vaccine = pd.read_csv("Clean_data/vaccine_us_county_clean.csv",low_memory=False)
fy_vaccine_df = fy_vaccine[['Date', 'FIPS','Series_Complete_Pop_Pct']].copy()
fy_vaccine_df.head()

,Date,FIPS,Series_Complete_Pop_Pct
0,09/30/2021,49057,47.1
1,09/30/2021,16027,34.4
2,09/30/2021,18171,40.4
3,09/30/2021,27007,50.1
4,09/30/2021,8041,52.6


In [3]:
# rename columns for vaccine data
fy_vaccine_df = fy_vaccine_df.rename(columns={"FIPS": "fips", "Series_Complete_Pop_Pct": "covid_vaccine_rate"})
fy_vaccine_df.head()

,Date,fips,covid_vaccine_rate
0,09/30/2021,49057,47.1
1,09/30/2021,16027,34.4
2,09/30/2021,18171,40.4
3,09/30/2021,27007,50.1
4,09/30/2021,8041,52.6


In [4]:
# change the data type for 'fips' from int to str
fy_vaccine_df = fy_vaccine_df.astype({"fips": str})

# if fips has only 4 numbers, add 0 to the begining to get 5 number str for consistancy
for i in range(len(fy_vaccine_df)):
    if len(fy_vaccine_df.iloc[i,1])==4:
        fy_vaccine_df.iloc[i,1] = f"0{fy_vaccine_df.iloc[i,1]}"
        
fy_vaccine_df.head()

,Date,fips,covid_vaccine_rate
0,09/30/2021,49057,47.1
1,09/30/2021,16027,34.4
2,09/30/2021,18171,40.4
3,09/30/2021,27007,50.1
4,09/30/2021,08041,52.6


In [5]:
# Get Sociohealth data from "02_us_county_sociohealth_data"
fy_soci_df = pd.read_csv("Resource/02_us_county_sociohealth_data.csv",low_memory=False)
fy_soci_df.head()

,fips,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,num_deaths,years_of_potential_life_lost_rate,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,01001,Alabama,Autauga,32.534928,-86.642748,55049,594.446120,92.605533,791.0,8128.591190,...,0.6339,0.5355,0.5976,0.6791,0.7268,0.2477,0.3298,0.1251,0.2881,0.3773
1,01003,Alabama,Baldwin,30.727489,-87.722575,199510,1589.807425,125.493187,2967.0,7354.122530,...,0.5253,0.5282,0.5294,0.9733,0.5387,0.2639,0.0872,0.3438,0.3324,0.2757
2,01005,Alabama,Barbour,31.869589,-85.393213,26614,884.875776,30.076538,472.0,10253.573403,...,0.9042,0.6979,0.8558,0.2814,0.9370,0.4438,0.8816,0.9427,0.9312,0.9847
3,01007,Alabama,Bibb,32.998634,-87.126480,22572,622.582355,36.255444,471.0,11977.539484,...,0.6450,0.3553,0.5018,0.4072,0.9249,0.0248,0.5645,0.9156,0.6663,0.5737
4,01009,Alabama,Blount,33.980878,-86.567383,57704,644.806508,89.490412,1085.0,11335.071134,...,0.4238,0.7482,0.5992,0.1344,0.8465,0.5056,0.1907,0.1515,0.1827,0.4986


In [6]:
# Merge vaccine data with Sociohealth data
fy_df = pd.merge(fy_vaccine_df, fy_soci_df, on = 'fips', how = 'outer')
fy_df.head()

,Date,fips,covid_vaccine_rate,state,county,lat,lon,total_population,area_sqmi,population_density_per_sqmi,...,percentile_rank_minorities,percentile_rank_limited_english_abilities,percentile_rank_minority_status_and_language_theme,percentile_rank_multi_unit_housing,percentile_rank_mobile_homes,percentile_rank_overcrowding,percentile_rank_no_vehicle,percentile_rank_institutionalized_in_group_quarters,percentile_rank_housing_and_transportation,percentile_rank_social_vulnerability
0,09/30/2021,49057,47.1,Utah,Weber,41.269832,-111.913401,241328.0,576.265130,418.779460,...,0.6141,0.8297,0.7714,0.8539,0.1391,0.7673,0.4734,0.2798,0.4897,0.4241
1,09/30/2021,16027,34.4,Idaho,Canyon,43.625133,-116.709318,202782.0,587.396831,345.221475,...,0.6883,0.8660,0.8313,0.5008,0.3973,0.8848,0.2340,0.4021,0.4374,0.7584
2,09/30/2021,18171,40.4,Indiana,Warren,40.346943,-87.353301,8309.0,364.681279,22.784279,...,0.0449,0.2219,0.0869,0.0742,0.2869,0.5791,0.0551,0.1764,0.0302,0.0637
3,09/30/2021,27007,50.1,Minnesota,Beltrami,47.973783,-94.937688,45644.0,2504.692890,18.223392,...,0.6689,0.0974,0.3623,0.8634,0.5667,0.6183,0.6934,0.8064,0.9478,0.7472
4,09/30/2021,08041,52.6,Colorado,El Paso,38.832110,-104.525471,665171.0,2126.840045,312.750835,...,0.6960,0.7202,0.7539,0.9252,0.1296,0.6778,0.2630,0.6148,0.5406,0.3642


In [7]:
# Save data into a csv file
fy_df.to_csv("Clean_data/fy_clean_data.csv", encoding="utf-8", index=False)

In [ ]:
#ricardo's code ends here